# 🐺 TEST BEFORE YOU BUILD
## The Right Way: Validate in Jupyter FIRST, then create .py scripts

**WRONG:** Build 40 wolf_*.py files without testing

**RIGHT:** Test ideas here, keep what works, delete what doesn't

---
## Setup: Import Libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded")

---
## TEST 1: Does This Edge Actually Work?
### Example: Monday AI Pattern - CIFR +4.39% avg?

In [ ]:
# Let's ACTUALLY test if CIFR moves up on Mondays

ticker = 'CIFR'
stock = yf.Ticker(ticker)
hist = stock.history(period='6mo')

# Filter for Mondays only
hist['DayOfWeek'] = hist.index.dayofweek  # 0 = Monday
mondays = hist[hist['DayOfWeek'] == 0].copy()

# Calculate Monday returns
mondays['Return'] = ((mondays['Close'] - mondays['Open']) / mondays['Open']) * 100

# Results
avg_return = mondays['Return'].mean()
win_rate = (mondays['Return'] > 0).sum() / len(mondays) * 100

print(f"📊 {ticker} MONDAY ANALYSIS (Last 6 months)")
print(f"   Total Mondays: {len(mondays)}")
print(f"   Average Monday Return: {avg_return:+.2f}%")
print(f"   Win Rate: {win_rate:.1f}%")
print(f"\n   Best Monday: {mondays['Return'].max():+.2f}%")
print(f"   Worst Monday: {mondays['Return'].min():+.2f}%")

# Show last 10 Mondays
print(f"\n📅 LAST 10 MONDAYS:")
for date, row in mondays.tail(10).iterrows():
    print(f"   {date.strftime('%Y-%m-%d')}: {row['Return']:+.2f}%")

### ✅ IF IT WORKS → Build the scanner
### ❌ IF IT DOESN'T → Don't waste time building it

---
## TEST 2: Quick Idea Validation
### Does TLRY actually have a catalyst right now?

In [ ]:
# You said TLRY moved - let's see what actually happened

ticker = 'TLRY'
stock = yf.Ticker(ticker)

# Recent performance
hist = stock.history(period='1mo')
current = hist['Close'].iloc[-1]
week_ago = hist['Close'].iloc[-5] if len(hist) >= 5 else hist['Close'].iloc[0]
month_ago = hist['Close'].iloc[0]

week_change = ((current - week_ago) / week_ago) * 100
month_change = ((current - month_ago) / month_ago) * 100

print(f"📊 {ticker} @ ${current:.2f}")
print(f"   Week: {week_change:+.1f}%")
print(f"   Month: {month_change:+.1f}%")

# Check for recent news
news = stock.news
if news:
    print(f"\n📰 RECENT NEWS:")
    for item in news[:3]:
        print(f"   - {item['title'][:60]}...")
else:
    print("\n⚠️  No recent news found")

# Earnings date
try:
    cal = stock.calendar
    if 'Earnings Date' in cal:
        print(f"\n📅 Next Earnings: {cal['Earnings Date']}")
except:
    pass

---
## TEST 3: Compare Multiple Tickers
### Which stock is actually the best setup RIGHT NOW?

In [ ]:
# Test multiple candidates at once

candidates = ['SOUN', 'BBAI', 'OUST', 'RCAT', 'SMR']

results = []

for ticker in candidates:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period='1mo')
        
        if len(hist) < 5:
            continue
        
        current = hist['Close'].iloc[-1]
        week_ago = hist['Close'].iloc[-5]
        week_change = ((current - week_ago) / week_ago) * 100
        
        # RSI
        delta = hist['Close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=14).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs.iloc[-1])) if loss.iloc[-1] != 0 else 50
        
        # Volume
        vol_ratio = hist['Volume'].iloc[-1] / hist['Volume'].mean()
        
        results.append({
            'Ticker': ticker,
            'Price': f'${current:.2f}',
            'Week %': f'{week_change:+.1f}%',
            'RSI': f'{rsi:.0f}',
            'Vol': f'{vol_ratio:.1f}x'
        })
    except:
        pass

# Display as table
df = pd.DataFrame(results)
print("\n📊 CANDIDATE COMPARISON:")
print(df.to_string(index=False))
print("\n✅ RSI < 65 = Good entry")
print("⚠️  RSI > 70 = Wait for pullback")

---
## TEST 4: Backtest an Edge
### Crash Bounce: Does buying -15% drops actually work?

In [ ]:
# Test crash bounce edge on WULF (69% win rate claimed)

ticker = 'WULF'
stock = yf.Ticker(ticker)
hist = stock.history(period='1y')

# Find crash days (-15% or more)
hist['Daily_Return'] = hist['Close'].pct_change() * 100
crashes = hist[hist['Daily_Return'] <= -15].copy()

print(f"📊 CRASH BOUNCE BACKTEST: {ticker}")
print(f"   Period: {hist.index[0].strftime('%Y-%m-%d')} to {hist.index[-1].strftime('%Y-%m-%d')}")
print(f"   Crash days (-15%+): {len(crashes)}")

if len(crashes) > 0:
    # Check what happened 3 days after each crash
    bounces = []
    
    for crash_date, crash_row in crashes.iterrows():
        crash_idx = hist.index.get_loc(crash_date)
        
        # Get price 3 days later (if exists)
        if crash_idx + 3 < len(hist):
            entry_price = crash_row['Close']
            exit_price = hist.iloc[crash_idx + 3]['Close']
            bounce_return = ((exit_price - entry_price) / entry_price) * 100
            
            bounces.append({
                'date': crash_date.strftime('%Y-%m-%d'),
                'crash': crash_row['Daily_Return'],
                'bounce': bounce_return,
                'win': bounce_return > 0
            })
    
    # Results
    if bounces:
        avg_bounce = np.mean([b['bounce'] for b in bounces])
        win_rate = sum([b['win'] for b in bounces]) / len(bounces) * 100
        
        print(f"\n📈 BOUNCE RESULTS (3-day hold):")
        print(f"   Average Bounce: {avg_bounce:+.2f}%")
        print(f"   Win Rate: {win_rate:.1f}%")
        
        print(f"\n📅 LAST 5 CRASHES:")
        for b in bounces[-5:]:
            win_emoji = "✅" if b['win'] else "❌"
            print(f"   {win_emoji} {b['date']}: Crash {b['crash']:.1f}% → Bounce {b['bounce']:+.1f}%")
        
        print(f"\n{'✅ EDGE VALIDATED' if win_rate > 60 else '❌ EDGE QUESTIONABLE'}")
else:
    print("   No crash days found in period")

---
## TEST 5: SEC Filings - Are They Actually Useful?
### Check if 8-K filings predict moves

In [ ]:
# Test: Did APLD move after its 8-K filing on Jan 7?

ticker = 'APLD'
filing_date = '2026-01-07'

stock = yf.Ticker(ticker)
hist = stock.history(period='1mo')

# Find filing date in history
filing_idx = None
for i, date in enumerate(hist.index):
    if date.strftime('%Y-%m-%d') == filing_date:
        filing_idx = i
        break

if filing_idx and filing_idx + 3 < len(hist):
    filing_price = hist.iloc[filing_idx]['Close']
    next_day = hist.iloc[filing_idx + 1]['Close']
    three_days = hist.iloc[filing_idx + 3]['Close']
    
    day1_return = ((next_day - filing_price) / filing_price) * 100
    day3_return = ((three_days - filing_price) / filing_price) * 100
    
    print(f"📋 {ticker} 8-K FILING: {filing_date}")
    print(f"   Filing Day Close: ${filing_price:.2f}")
    print(f"   Next Day: ${next_day:.2f} ({day1_return:+.1f}%)")
    print(f"   3 Days Later: ${three_days:.2f} ({day3_return:+.1f}%)")
    
    if day3_return > 10:
        print(f"\n✅ 8-K WAS A STRONG SIGNAL (+{day3_return:.1f}%)")
    elif day3_return > 0:
        print(f"\n🟡 8-K was okay signal (+{day3_return:.1f}%)")
    else:
        print(f"\n❌ 8-K was not a good signal ({day3_return:.1f}%)")
else:
    print(f"⚠️  Filing date not found in recent history")

---
## THE WORKFLOW

### 1. Have an idea → TEST IT HERE
### 2. If it works → Create a .py script in /core/
### 3. If it doesn't → Delete the cell and move on

**DON'T:** Create 40 wolf_*.py files without testing

**DO:** Validate in Jupyter, keep only what works

---
## 🐺 YOUR TESTING SPACE
### Use cells below to test YOUR ideas before building

In [ ]:
# TEST YOUR IDEA HERE
# Does it work? Yes → Build it. No → Delete this cell.